In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import datetime

# --- 1. 데이터 임포트 ---
from services.tables.HR_Core.job_info_table import job_info_df
from services.tables.HR_Core.basic_info_table import emp_df
from services.tables.HR_Core.department_info_table import department_info_df
from services.tables.HR_Core.department_table import (
    dept_level_map, parent_map_dept, dept_name_map,
    division_order, office_order
)
from services.helpers.utils import find_parents

def create_figure():
    """
    제안 9: 조직 활력도 진단 (연도별 직무 이동률) 그래프를 생성합니다.
    """
    # --- 2. 데이터 준비 및 가공 ---
    # 2-1. 연도별/조직별 직무 변경 횟수 집계 (Numerator)
    job_changes = job_info_df.copy()
    job_changes = pd.merge(job_changes, emp_df[['EMP_ID', 'IN_DATE']], on='EMP_ID', how='left')
    job_changes = job_changes[job_changes['JOB_APP_START_DATE'] > job_changes['IN_DATE']]
    job_changes['YEAR'] = job_changes['JOB_APP_START_DATE'].dt.year
    
    dept_info_sorted = department_info_df.sort_values(['DEP_APP_START_DATE', 'EMP_ID'])
    job_changes_with_dept = pd.merge_asof(
        job_changes.sort_values('JOB_APP_START_DATE'), 
        dept_info_sorted[['EMP_ID', 'DEP_APP_START_DATE', 'DEP_ID']],
        left_on='JOB_APP_START_DATE', right_on='DEP_APP_START_DATE', by='EMP_ID', direction='backward'
    )
    parent_info = job_changes_with_dept['DEP_ID'].apply(lambda x: find_parents(x, dept_level_map, parent_map_dept, dept_name_map))
    job_changes_with_dept = pd.concat([job_changes_with_dept, parent_info], axis=1)
    job_changes_with_dept = job_changes_with_dept.dropna(subset=['DIVISION_NAME', 'OFFICE_NAME'])
    
    changes_by_div_year = job_changes_with_dept.groupby(['YEAR', 'DIVISION_NAME'], observed=False).size().reset_index(name='CHANGE_COUNT')
    changes_by_office_year = job_changes_with_dept.groupby(['YEAR', 'DIVISION_NAME', 'OFFICE_NAME'], observed=False).size().reset_index(name='CHANGE_COUNT')

    # 2-2. 연도별/조직별 평균 재직자 수 계산 (Denominator)
    analysis_years = sorted(job_changes_with_dept['YEAR'].unique())
    headcount_records = []
    
    def get_headcount_by_org_at_date(target_date):
        active_emps = emp_df[(emp_df['IN_DATE'] <= target_date) & (emp_df['OUT_DATE'].isnull() | (emp_df['OUT_DATE'] >= target_date))]
        if active_emps.empty: return pd.DataFrame()
        active_depts = pd.merge_asof(
            active_emps[['EMP_ID', 'IN_DATE']].sort_values('IN_DATE'),
            department_info_df.sort_values('DEP_APP_START_DATE'),
            left_on='IN_DATE', right_on='DEP_APP_START_DATE', by='EMP_ID', direction='backward'
        )
        parent_info_active = active_depts['DEP_ID'].apply(lambda x: find_parents(x, dept_level_map, parent_map_dept, dept_name_map))
        active_depts = pd.concat([active_depts, parent_info_active], axis=1)
        return active_depts.dropna(subset=['DIVISION_NAME', 'OFFICE_NAME'])

    for year in analysis_years:
        start_of_year, end_of_year = pd.to_datetime(f'{year}-01-01'), pd.to_datetime(f'{year}-12-31')
        headcount_start_df = get_headcount_by_org_at_date(start_of_year)
        headcount_end_df = get_headcount_by_org_at_date(end_of_year)
        count_start_div = headcount_start_df.groupby('DIVISION_NAME', observed=False).size()
        count_end_div = headcount_end_df.groupby('DIVISION_NAME', observed=False).size()
        avg_headcount_div = ((count_start_div.add(count_end_div, fill_value=0)) / 2).reset_index(name='HEADCOUNT')
        avg_headcount_div['YEAR'] = year
        count_start_office = headcount_start_df.groupby(['DIVISION_NAME', 'OFFICE_NAME'], observed=False).size()
        count_end_office = headcount_end_df.groupby(['DIVISION_NAME', 'OFFICE_NAME'], observed=False).size()
        avg_headcount_office = ((count_start_office.add(count_end_office, fill_value=0)) / 2).reset_index(name='HEADCOUNT')
        avg_headcount_office['YEAR'] = year
        headcount_records.extend([avg_headcount_div, avg_headcount_office])
    headcount_df = pd.concat(headcount_records, ignore_index=True)
    
    # 2-3. 데이터 통합 및 이동률 계산
    div_analysis_df = pd.merge(changes_by_div_year, headcount_df.dropna(subset=['DIVISION_NAME']), on=['YEAR', 'DIVISION_NAME'], how='left')
    office_analysis_df = pd.merge(changes_by_office_year, headcount_df.dropna(subset=['OFFICE_NAME']), on=['YEAR', 'DIVISION_NAME', 'OFFICE_NAME'], how='left')
    for df in [div_analysis_df, office_analysis_df]:
        df['MOBILITY_RATE'] = (df['CHANGE_COUNT'] / df['HEADCOUNT'] * 100).fillna(0)

    # --- 3. Plotly 인터랙티브 그래프 생성 ---
    y_max = pd.concat([div_analysis_df['MOBILITY_RATE'], office_analysis_df['MOBILITY_RATE']]).max()
    fixed_y_range = [0, y_max * 1.15]
    
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly
    
    for i, div_name in enumerate(division_order):
        df_filtered = div_analysis_df[div_analysis_df['DIVISION_NAME'] == div_name].sort_values('YEAR')
        if not df_filtered.empty:
            fig.add_trace(go.Scatter(x=df_filtered['YEAR'], y=df_filtered['MOBILITY_RATE'], mode='lines+markers+text', name=div_name, 
                                     line=dict(color=colors[i]), text=df_filtered['MOBILITY_RATE'].round(2).astype(str) + '%', textposition='top center'))

    office_traces_map = {}
    trace_idx_counter = len(fig.data)
    for div_name in division_order:
        office_div_df = office_analysis_df[office_analysis_df['DIVISION_NAME'] == div_name]
        offices_in_div_sorted = [o for o in office_order if o in office_div_df['OFFICE_NAME'].unique()]
        office_traces_map[div_name] = []
        for j, office_name in enumerate(offices_in_div_sorted):
            df_filtered = office_div_df[office_div_df['OFFICE_NAME'] == office_name].sort_values('YEAR')
            if not df_filtered.empty:
                fig.add_trace(go.Scatter(x=df_filtered['YEAR'], y=df_filtered['MOBILITY_RATE'], mode='lines+markers+text', name=office_name, visible=False, 
                                         line=dict(color=colors[j % len(colors)]), text=df_filtered['MOBILITY_RATE'].round(2).astype(str) + '%', textposition='top center'))
                office_traces_map[div_name].append(trace_idx_counter)
                trace_idx_counter += 1
    
    # --- 4. 드롭다운 메뉴 및 레이아웃 업데이트 ---
    buttons = [dict(label='전체 (Division 보기)', method='update', args=[{'visible': [True]*len(division_order) + [False]*(len(fig.data)-len(division_order))}])]
    for div_name in division_order:
        visibility_mask = [False] * len(fig.data)
        for trace_idx in office_traces_map.get(div_name, []):
            visibility_mask[trace_idx] = True
        buttons.append(dict(label=f'{div_name} 상세', method='update', args=[{'visible': visibility_mask}]))
    
    fig.update_layout(
        updatemenus=[dict(active=0, buttons=buttons, direction="down", pad={"r": 10, "t": 10}, showactive=True, x=0.01, xanchor="left", y=1.1, yanchor="top")],
        title_text='연도별/조직별 직무 이동률(%) 변화 추이',
        xaxis_title='연도', yaxis_title='직무 이동률 (%)',
        font_size=14, height=700,
        legend_title_text='조직',
        xaxis=dict(type='category'), 
        yaxis=dict(ticksuffix="%", range=fixed_y_range)
    )
    
    return fig

# 이 파일을 직접 실행할 경우 그래프를 생성하여 보여줍니다.
if __name__ == '__main__':
    pio.renderers.default = 'browser'
    fig = create_figure()
    fig.show()